In [1]:
import os
import sys
# 如果当前代码文件运行测试需要加入修改路径，避免出现后导包问题
BASE_DIR = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.insert(0, os.path.join(BASE_DIR))
PYSPARK_PYTHON = "/opt/anaconda3/envs/rec_sys/bin/python"

# 当存在多个版本时，不指定很可能会导致出错
os.environ["PYSPARK_PYTHON"] = PYSPARK_PYTHON
os.environ["PYSPARK_DRIVER_PYTHON"] = PYSPARK_PYTHON

from offline import SparkSessionBase

In [2]:
class KeywordsToTfidf(SparkSessionBase):
    
    SPARK_APP_NAME = "keywordsByTFIDF"
    SPARK_EXECUTOR_MEMORY="4g"

    
    def __init__(self):
        self.spark = self._create_spark_session()

In [3]:
ktt = KeywordsToTfidf()

In [12]:
ktt.spark.stop()

In [4]:
ktt.spark.sql("use article")
article_data = ktt.spark.sql("select * from article_data limit 10")

In [5]:
import codecs
global stopwords_list
stopwords_path = "/home/data/etc/stop_words.txt"
stopwords_list = [i.strip() for i in codecs.open(stopwords_path).readlines()]

In [6]:
def segmentation(partition):
    import jieba
    import os
    import jieba.posseg as pseg
    import re
    import codecs
     
    # 获取停用词
#     def get_stopwords_list():
#         """返回stopwords列表"""
#         stopwords_path = "/home/data/etc/stop_words.txt"
#         stopwords_list = [i.strip() for i in codecs.open(stopwords_path,'w').readlines()]
        
#     # 所有的停用词列表
#     stopwords_list = get_stopwords_list()
    
    # 分词
    def cut_sentence(sentence):
        """对切割之后的词语进行过滤，去除停用词，保留名词，英文和自定义词库中的词，长度大于2的词"""
        # print(sentence,"*"*100)
        # eg:[pair('今天', 't'), pair('有', 'd'), pair('雾', 'n'), pair('霾', 'g')]
        seg_list = pseg.lcut(sentence)
        seg_list = [i for i in seg_list if i.flag not in stopwords_list]
        filtered_words_list = []
        for seg in seg_list:
            # print(seg)
            if len(seg.word) <= 1:
                continue
            elif seg.flag == "eng":
                if len(seg.word) <= 2:
                    continue
                else:
                    filtered_words_list.append(seg.word)
            elif seg.flag.startswith("n"):
                filtered_words_list.append(seg.word)
            elif seg.flag in ["x", "eng"]:  # 是自定一个词语或者是英文单词
                filtered_words_list.append(seg.word)
        return filtered_words_list
    
    def cut_sentence_2(sentence):
        """对切割之后的词语进行过滤，去除停用词，保留名词，英文和自定义词库中的词，长度大于2的词"""
        # print(sentence,"*"*100)
        # eg:[pair('今天', 't'), pair('有', 'd'), pair('雾', 'n'), pair('霾', 'g')]
        postag = ['nr','ns','nt','nx','nz','n']
        pseg = pkuseg.pkuseg(postag = True)
        seg_list = pseg.cut(sentence)
        seg_list = [i for i in seg_list if i.flag not in stopwords_list]
        filtered_words_list = []
        for seg in seg_list:
            try:
                if len(seg[0]) <= 1:
                    continue
                elif seg[-1] in postag:
                    if len(seg[0]) <= 2:
                        continue
                    else:
                        filtered_words_list.append(seg[0])
            except Exception as e:
                print("you have a error is {},the data is {}".format(e, seg))
        return filtered_words_list
    
    for row in partition:
        sentence = re.sub("<.*?>", "", row.sentence)    # 替换掉标签数据
        words = cut_sentence(sentence)
        yield row.article_id, row.channel_name, words

In [12]:
article_data.show()

+--------------------+------------+----------------------------------------+-------------------------------------+------------------------+-----------------------------+-----------------------------+
|          article_id|channel_name|                                   title|                              content|                  source|                  source_tags|                     sentence|
+--------------------+------------+----------------------------------------+-------------------------------------+------------------------+-----------------------------+-----------------------------+
|00000982fda1948f4...|      投资界|     新能源汽车10大趋势预测：商用车率...|编者按：能源危机愈演愈烈，将新能源...|微信公众号：品途商业评论|    [ "汽车" , "新能源" , ...|    [ "汽车" , "新能源" , ...|
|000039bca765fca66...|      猎云网|    成都监察委立案调查东方广益6亿投资...| 猎云网：成都成华区监察委立案调查“...|                  unknow|  [ "罗永浩" , "锤子科技" ...|  [ "罗永浩" , "锤子科技" ...|
|0000b6f5182e097e8...|      砍柴网|      腾不出手来管理二手车业务   特斯...|    11月20日消息，据两位知情人士透...|                  unkn

In [13]:
words_df = article_data.rdd.mapPartitions(segmentation).toDF(["article_id", "channel_name", "words"])

In [14]:
words_df.registerTempTable("templetable")

In [15]:
ktt.spark.sql("select * from templetable limit 10").show()

+--------------------+------------+-----------------------------+
|          article_id|channel_name|                        words|
+--------------------+------------+-----------------------------+
|0ae5ec196aa06c47b...|      品途网| [阿里, 智能, 汽车, 品途, ...|
|0ae61c67f4510fc0d...|      DoNews|         [unknow, DoNews, ...|
|0ae64ae3d5efd8db2...|      鞭牛士| [神舟, 牛士, 倾心, 神舟, ...|
|0ae75ea628ffdc7ee...|        36氪|    [unknow, 埃森哲, 鼻祖,...|
|0ae7a7499baea87bf...|    站长之家| [斗鱼, 宣告, 斗鱼, 家暴, ...|
|0ae7f35879dca0207...|      品途网| [手游, 游戏, 公司, 魔兽, ...|
|0ae87d308dc7f2229...|      思达派|[思达, 顶级, 机构, 专业化,...|
|0ae89a55248c00603...|        亿欧|   [ofo, 摩拜, 戴威, 黄车,...|
|0ae8deaf40fcaab1d...| HRTechChina|[资讯, 人力资源, 实验室, L...|
|0ae8e8459a65817b6...|  世界经理人|[哲学, 概念, 企业家, 企业,...|
+--------------------+------------+-----------------------------+



In [16]:
from pyspark.ml.feature import CountVectorizer

In [ ]:
cv = CountVectorizer(inputCol="words", outputCol="countFeatures", vocabSize=200*10000, minDF=1.0)
# 训练词频统计模型
cv_model = cv.fit(words_df)
cv_model.write().overwrite().save("hdfs://192.168.0.52:8020/models/CV.model")

In [4]:
from pyspark.ml.feature import CountVectorizerModel
cv_model = CountVectorizerModel.load("hdfs://192.168.0.52:8020/models/CV.model")
# 得出词频向量结果
cv_result = cv_model.transform(words_df)
# 训练IDF模型
from pyspark.ml.feature import IDF
idf = IDF(inputCol="countFeatures", outputCol="idfFeatures")
idfModel = idf.fit(cv_result)
idfModel.write().overwrite().save("hdfs://192.168.0.52:8020/models/IDF.model")

NameError: name 'words_df' is not defined

In [24]:
idfModel.idf.toArray()[:20]

array([0.37702786, 0.61473956, 0.50244703, 0.63056421, 0.72375587,
       0.61016019, 0.68230305, 0.70786613, 0.62147479, 0.64961663,
       0.63001098, 0.94416523, 0.84821951, 1.17245272, 0.99504718,
       0.77097374, 0.96936827, 1.35487223, 0.73713712, 1.69034085])

In [25]:
cv_result.show()

+--------------------+------------+------------------------------+--------------------+
|          article_id|channel_name|                         words|       countFeatures|
+--------------------+------------+------------------------------+--------------------+
|00000982fda1948f4...|      投资界|[汽车, 新能源, 汽车, 投资界...|(572577,[0,1,2,3,...|
|000039bca765fca66...|      猎云网| [罗永浩, 锤子, 科技, 广益,...|(572577,[0,1,2,5,...|
|0000b6f5182e097e8...|      砍柴网|[特斯拉, 二手车, 业务, 砍柴...|(572577,[0,1,2,12...|
|0000bca75b0386b06...|      希鸥网|      [unknow, 希鸥, 用户, ...|(572577,[0,1,2,3,...|
|0000f7b314fe7970d...|        36氪|     [unknow, 谷歌, 母公司,...|(572577,[0,4,5,6,...|
|00019c1868a4ae670...|      投资界|[雅居乐, 雅尊会, 投资界, 经...|(572577,[1,3,7,13...|
|0001a643f8a623a0c...|      希鸥网|      [unknow, 希鸥, 希鸥, ...|(572577,[0,1,2,3,...|
|0001c67ae9c818e47...|      猎云网|      [Journal, 信息, 工具,...|(572577,[0,4,5,7,...|
|0002d00a0273e73c8...|        亿欧|[公立医院, 医生, 执业, 黄石...|(572577,[0,1,2,3,...|
|000314168bdf88130...|      投资界|  

In [33]:
tfidf_res = idfModel.transform(cv_result)

In [34]:
tfidf_res.show()

+--------------------+------------+------------------------------+--------------------+--------------------+
|          article_id|channel_name|                         words|       countFeatures|         idfFeatures|
+--------------------+------------+------------------------------+--------------------+--------------------+
|00000982fda1948f4...|      投资界|[汽车, 新能源, 汽车, 投资界...|(572577,[0,1,2,3,...|(572577,[0,1,2,3,...|
|000039bca765fca66...|      猎云网| [罗永浩, 锤子, 科技, 广益,...|(572577,[0,1,2,5,...|(572577,[0,1,2,5,...|
|0000b6f5182e097e8...|      砍柴网|[特斯拉, 二手车, 业务, 砍柴...|(572577,[0,1,2,12...|(572577,[0,1,2,12...|
|0000bca75b0386b06...|      希鸥网|      [unknow, 希鸥, 用户, ...|(572577,[0,1,2,3,...|(572577,[0,1,2,3,...|
|0000f7b314fe7970d...|        36氪|     [unknow, 谷歌, 母公司,...|(572577,[0,4,5,6,...|(572577,[0,4,5,6,...|
|00019c1868a4ae670...|      投资界|[雅居乐, 雅尊会, 投资界, 经...|(572577,[1,3,7,13...|(572577,[1,3,7,13...|
|0001a643f8a623a0c...|      希鸥网|      [unknow, 希鸥, 希鸥, ...|(572577,[0,1,2,3,...|(57

In [27]:
keywords_list_with_idf = list(zip(cv_model.vocabulary, idfModel.idf.toArray()))

In [29]:
def func(data):
    for index in range(len(data)):
        data[index] = list(data[index])
        data[index].append(index)
        data[index][1] = float(data[index][1])
func(keywords_list_with_idf)
sc = ktt.spark.sparkContext
rdd = sc.parallelize(keywords_list_with_idf)
df = rdd.toDF(["keywords", "idf", "index"])
df.write.insertInto('idf_keywords_values')

In [42]:
ktt.spark.sql("use article")

DataFrame[]

In [44]:
ktt.spark.sql("select * from idf_keywords_values limit 10").show()

+-------+------------------+-----+
|keyword|               idf|index|
+-------+------------------+-----+
|   劲邦| 9.091431221721468|71680|
|   项兵| 9.576939037503168|71681|
|   龙巅| 11.25091547107484|71682|
|   SoIC|10.403617610687636|71683|
| Ginger| 9.305005322019527|71684|
| 市议员| 9.130651934874749|71685|
|   手力| 8.982231929756477|71686|
| fitbit| 9.305005322019527|71687|
|   终章| 9.017323249567745|71688|
| 罗忠生|  9.71047043012769|71689|
+-------+------------------+-----+



In [45]:
ktt.spark.sql(" desc idf_keywords_values").show()

+--------+---------+----------+
|col_name|data_type|   comment|
+--------+---------+----------+
| keyword|   string|article_id|
|     idf|   double|       idf|
|   index|      int|     index|
+--------+---------+----------+



In [49]:
def func(partition):
    TOPK = 20
    for row in partition:
        # 找到索引与IDF值并进行排序
        _ = list(zip(row.idfFeatures.indices, row.idfFeatures.values))
        _ = sorted(_, key=lambda x: x[1], reverse=True)
        result = _[:TOPK]
        for word_index, tfidf in result:
            yield row.article_id, row.channel_name, int(word_index), round(float(tfidf), 4)

In [50]:
_keywordsByTFIDF = tfidf_res.rdd.mapPartitions(func).toDF(["article_id", "channel_name", "index", "tfidf"])

In [59]:
_keywordsByTFIDF.write.insertInto("pre_tfidf_keywords_values")

In [57]:
keywordsIndex.show()

+-------+---+
|keyword|idx|
+-------+---+
|   公司|  0|
|   企业|  1|
|   市场|  2|
|   中国|  3|
|   用户|  4|
|   产品|  5|
|   技术|  6|
|   平台|  7|
|   行业|  8|
|   数据|  9|
|   问题| 10|
| 互联网| 11|
|   业务| 12|
|   品牌| 13|
|   科技| 14|
|   领域| 15|
|   内容| 16|
|   智能| 17|
|   时间| 18|
|   腾讯| 19|
+-------+---+
only showing top 20 rows



In [51]:
keywordsIndex = ktt.spark.sql("select keyword, index idx from idf_keywords_values")
keywordsByTFIDF = _keywordsByTFIDF.join(keywordsIndex, keywordsIndex.idx == _keywordsByTFIDF.index).select(["article_id", "channel_name", "keyword", "tfidf"])

In [58]:
keywordsByTFIDF.show()

+--------------------+------------+-------+-------+
|          article_id|channel_name|keyword|  tfidf|
+--------------------+------------+-------+-------+
|0afd1b0d983d1c257...|        36氪|   全球| 1.0243|
|0b2009b79fb2eb230...|        36氪|   全球| 5.1217|
|0b4e4c2254c54a914...|      品途网|   全球|22.5355|
|0b57598f2de7f168c...|        亿欧|   全球|15.3651|
|0b5e51cdabe3d6052...|        亿欧|   全球|12.2921|
|0b7143b66475c6d48...|      品途网|   全球|15.3651|
|0b7af754981dbba09...|      投资界|   全球|13.3165|
|0ba08990543798a16...|      品途网|   全球|15.3651|
|0bb7238085dbc11ce...|    IT时代网|   全球|17.4138|
|0bbb69d12fe2fa4a6...|      DoNews|   全球| 7.1704|
|0bd19c2d53e8dc7d0...|    IT时代网|   全球| 9.2191|
|0bf34838372b3bb82...|      品途网|   全球|15.3651|
|0c331b2c6d47aa4d0...|      IT之家|   全球| 9.2191|
|0c47d47456d2e76df...|      鞭牛士|   全球|12.2921|
|0c4f9d99965c834db...|      DoNews|   全球| 2.0487|
|0c54f8a36ad23b53c...|      品途网|   全球|14.3408|
|0c694ba7d9f0b88c6...|      砍柴网|   全球|20.4869|
|0c72b2f824ff58b30...|        亿欧

In [61]:
keywordsByTFIDF.write.insertInto("tfidf_keywords_values")

In [7]:
ktt.spark.sql("desc tfidf_keywords_values").show()

+------------+---------+------------+
|    col_name|data_type|     comment|
+------------+---------+------------+
|  article_id|   string|  article_id|
|channel_name|   string|channel_name|
|     keyword|   string|     keyword|
|       tfidf|   double|       tfidf|
|article_time|   string|        null|
+------------+---------+------------+



In [8]:
ktt.spark.sql("select count(*) from tfidf_keywords_values ").show()

+--------+
|count(1)|
+--------+
| 4870934|
+--------+



In [9]:
ktt.spark.sql("select * from tfidf_keywords_values limit 10").show()

+--------------------+------------+--------+-------+-------------------+
|          article_id|channel_name| keyword|  tfidf|       article_time|
+--------------------+------------+--------+-------+-------------------+
|2701f73f7c9437176...|        36氪|阿里巴巴| 9.2886|2019-12-11 08:31:05|
|6c2f9195ecc86cf94...|      IT之家|    信用| 3.1126|2019-12-11 08:24:34|
|64cd91c52de06503e...|        36氪|     app|15.3317|2019-12-11 08:06:05|
|efb302b2dbbc29c20...|        亿欧|    低温|45.3093|2019-12-11 08:58:36|
|d7798752d5f82aad1...|      猎云网|    复星|29.1006|2019-12-10 19:58:11|
|d7798752d5f82aad1...|      猎云网|    复星|29.1006|2019-12-10 19:58:11|
|d7798752d5f82aad1...|      猎云网|    复星|29.1006|2019-12-10 19:58:11|
|d7798752d5f82aad1...|      猎云网|    复星|29.1006|2019-12-10 19:58:11|
|d7798752d5f82aad1...|      猎云网|    复星|29.1006|2019-12-10 19:58:11|
|d7798752d5f82aad1...|      猎云网|    复星|29.1006|2019-12-10 19:58:11|
+--------------------+------------+--------+-------+-------------------+



In [20]:
source_tags = ktt.spark.sql("select article_id as article_id2, channel_name as channel_name2, source_tags, article_time from article_data")

In [21]:
merge_keywords  = ktt.spark.sql("select article_id, min(channel_name) channel_name, collect_list(keyword) keywords, collect_list(tfidf) weights from tfidf_keywords_values group by article_id")

In [22]:
keywords_info = merge_keywords.join(source_tags, merge_keywords.article_id == source_tags.article_id2).select(["article_id", "channel_name", "keywords", "weights","source_tags","article_time"])

In [23]:
keywords_info.schema

StructType(List(StructField(article_id,StringType,true),StructField(channel_name,StringType,true),StructField(keywords,ArrayType(StringType,true),true),StructField(weights,ArrayType(DoubleType,true),true),StructField(source_tags,StringType,true),StructField(article_time,StringType,true)))

In [24]:
from pyspark.sql.functions import udf
import pyspark.sql.functions as F
from pyspark.sql.types import *,

In [59]:
@udf("string", StringType())
def change_unknow_udf(keywords, source_tags):
    if source_tags == "unknow":
        source_tags = keywords
    elif source_tags == None :
        source_tags = keywords
    elif len(source_tags) <=3:
        source_tags = keywords
    return source_tags

In [60]:
keywords_info = keywords_info.withColumn("topics", change_unknow_udf(keywords_info.keywords, keywords_info.source_tags))

In [61]:
keywords_info.show()

+--------------------+------------+------------------------------+--------------------+----------------------------+-------------------+------------------------------+
|          article_id|channel_name|                      keywords|             weights|                 source_tags|       article_time|                        topics|
+--------------------+------------+------------------------------+--------------------+----------------------------+-------------------+------------------------------+
|00a32656f423c19bd...|        36氪| [信心, 负面, 目标, 转化率,...|[19.6395, 24.0735...|                        null|2019-06-27 20:31:05| [信心, 负面, 目标, 转化率,...|
|00a77a605fa1f8307...|        亿欧|[板块, 学校, 普华永道, 香港...|[34.4602, 15.7381...|    [ "教育" , "政策" , "...|2019-01-21 18:30:00|      [ "教育" , "政策" , "...|
|0167a23fb8747af19...|        36氪|[上柜, 仿真技术, 折纸, 科技...|[9.305, 7.64, 8.3...|                        null|2019-03-14 13:44:56|[上柜, 仿真技术, 折纸, 科技...|
|016ba612ee61ec97f...|      DoNews|          [K11, C

In [ ]:
def _func(row):
    arry_topics = row.topics.replace("[","").replace("]","").replace("\"","").replace(" ","").split(",")
    return row.article_id, row.channel_name, dict(zip(row.keywords, row.weights)), arry_topics, row.article_time
article_profile = keywords_info.rdd.map(_func).toDF(["article_id", "channel_name", "keywords", "topics","article_time"])

In [ ]:
article_profile.schame

In [39]:
article_profile = article_profile.withColumn("topics", article_profile.topics)

AnalysisException: "cannot resolve '`topics`' due to data type mismatch: cannot cast string to array<string>;;\n'Project [article_id#314, channel_name#315, keywords#316, cast(topics#317 as array<string>) AS topics#375, article_time#318]\n+- LogicalRDD [article_id#314, channel_name#315, keywords#316, topics#317, article_time#318], false\n"

In [34]:
article_profile.schema

StructType(List(StructField(article_id,StringType,true),StructField(channel_name,StringType,true),StructField(keywords,MapType(StringType,DoubleType,true),true),StructField(topics,StringType,true),StructField(article_time,StringType,true)))

In [33]:
article_profile.write.insertInto("article_profile")

AnalysisException: "cannot resolve '`topics`' due to data type mismatch: cannot cast string to array<string>;;\n'InsertIntoHiveTable `article`.`article_profile`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, false, false, [article_id, channel_name, keyword, topics, article_time]\n+- 'Project [cast(article_id#314 as string) AS article_id#370, cast(channel_name#315 as string) AS channel_name#371, cast(keywords#316 as map<string,double>) AS keyword#372, cast(topics#317 as array<string>) AS topics#373, article_time#318]\n   +- LogicalRDD [article_id#314, channel_name#315, keywords#316, topics#317, article_time#318], false\n"

In [184]:
article_profile.show()

+--------------------+------------+-------------------------+------------------------------+
|          article_id|channel_name|                 keywords|                        topics|
+--------------------+------------+-------------------------+------------------------------+
|00a32656f423c19bd...|        36氪|[波斯 -> 31.87, 讲故事...|[销售额, 讲故事, 信心, 目标...|
|00a77a605fa1f8307...|        亿欧|   [A股 -> 30.4946, 庞...|      [ "教育" , "政策" , "...|
|0167a23fb8747af19...|        36氪|  [科技 -> 11.9406, 智...|  [助力, 场景, 画像, 杨恒, ...|
|016ba612ee61ec97f...|      DoNews|  [文化展 -> 9.4051, B...|    [体验, 芭比, K11, 布朗,...|
|01abaff78180a689a...|      投资界|  [大脑 -> 12.9711, 订...|      [ "海尔" , "蒙牛" , "...|
|01ad671ef2b30d864...|  世界经理人| [单兵作战 -> 13.5171,...|      [ "战略" , "绩效" , "...|
|01fc229aad630a827...|      鞭牛士|   [权限 -> 4.0537, Ap...|                 [ "用户信息"]|
|0201180aa7a6ea0b0...|        36氪|  [效应 -> 30.7139, 战...|     [效应, Leader, 延展性,...|
|020d506efcf0cb0d7...|        36氪|    [App -> 5.3512, 公...|    

In [186]:
ktt.spark.sql('select * from article_profile limit 1').show()

+--------------------+------------+-----------------------+-----------------------------+
|          article_id|channel_name|                keyword|                       topics|
+--------------------+------------+-----------------------+-----------------------------+
|00a32656f423c19bd...|        36氪|[讲故事 -> 18.5448, ...|[对象, 情感, 时候, 转化率,...|
+--------------------+------------+-----------------------+-----------------------------+

